# SkyInsight: Predictive Analytics for Cost-Effective Air Travel"


---

### Introduction

---

### 1.2 Data Collection

In [25]:
import pandas as pd

In [26]:
# data url = 'https://www.kaggle.com/datasets/shubhambathwal/flight-price-prediction/data'

In [27]:
df = pd.read_csv('../data/raw_dataset.csv')
df.drop(columns=['Unnamed: 0', 'days_left'], inplace=True)
df.head()

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,price
0,SpiceJet,SG-8709,Delhi,Evening,zero,Night,Mumbai,Economy,2.17,5953
1,SpiceJet,SG-8157,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.33,5953
2,AirAsia,I5-764,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,5956
3,Vistara,UK-995,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.25,5955
4,Vistara,UK-963,Delhi,Morning,zero,Morning,Mumbai,Economy,2.33,5955


---

### 1.3 Data Cleaning

In [28]:
df["price"] = round(df["price"] * 0.12 / 10, 2)
df["duration"] = round(df["duration"] * 60, 0).astype(int)
df["stops"].replace({"zero": 0, "one": 1, "two_or_more": 2}, inplace=True)
df["class"].replace({"Economy": 0, "Business": 1}, inplace=True)
df.rename(columns={'source_city': 'origin', 'destination_city': 'destination'}, inplace=True)

In [29]:
df.to_csv('../data/clean_dataset.csv', index=False)

In [30]:
df.isnull().sum()

airline           0
flight            0
origin            0
departure_time    0
stops             0
arrival_time      0
destination       0
class             0
duration          0
price             0
dtype: int64

In [31]:
df.head()

,airline,flight,origin,departure_time,stops,arrival_time,destination,class,duration,price
0,SpiceJet,SG-8709,Delhi,Evening,0,Night,Mumbai,0,130,71.44
1,SpiceJet,SG-8157,Delhi,Early_Morning,0,Morning,Mumbai,0,140,71.44
2,AirAsia,I5-764,Delhi,Early_Morning,0,Early_Morning,Mumbai,0,130,71.47
3,Vistara,UK-995,Delhi,Morning,0,Afternoon,Mumbai,0,135,71.46
4,Vistara,UK-963,Delhi,Morning,0,Morning,Mumbai,0,140,71.46


### 1.4 Data Transformation

In [32]:
df['duration'].describe()


count    300153.000000
mean        733.259055
std         431.518381
min          50.000000
25%         410.000000
50%         675.000000
75%         970.000000
max        2990.000000
Name: duration, dtype: float64

In [33]:
#feature engineering
#new feature for the time of day based based of the depature_time

#specify the range for the labels based on the quartiles
duration_bin_edges = [50, 410, 675, 970, 2990] 

#define label for each bin
duration_labels = ['Short', 'Medium', 'Long', 'Very Long']

#create the duration range feature
df['duration_range'] = pd.cut(df['duration'], 
                              bins= duration_bin_edges,
                             labels=duration_labels,
                             include_lowest=True)

In [34]:
#one hot encoding

df = pd.get_dummies(df, columns=['departure_time'], prefix='dept_time', dtype=int)

,airline,flight,origin,stops,arrival_time,destination,class,duration,price,duration_range,dept_time_Afternoon,dept_time_Early_Morning,dept_time_Evening,dept_time_Late_Night,dept_time_Morning,dept_time_Night
0,SpiceJet,SG-8709,Delhi,0,Night,Mumbai,0,130,71.44,Short,0,0,1,0,0,0
1,SpiceJet,SG-8157,Delhi,0,Morning,Mumbai,0,140,71.44,Short,0,1,0,0,0,0
2,AirAsia,I5-764,Delhi,0,Early_Morning,Mumbai,0,130,71.47,Short,0,1,0,0,0,0
3,Vistara,UK-995,Delhi,0,Afternoon,Mumbai,0,135,71.46,Short,0,0,0,0,1,0
4,Vistara,UK-963,Delhi,0,Morning,Mumbai,0,140,71.46,Short,0,0,0,0,1,0


In [36]:
df = pd.get_dummies(df, columns=['duration_range'],
                    prefix='trip_length', dtype=int)

In [37]:
df.head()

,airline,flight,origin,stops,arrival_time,destination,class,duration,price,dept_time_Afternoon,dept_time_Early_Morning,dept_time_Evening,dept_time_Late_Night,dept_time_Morning,dept_time_Night,trip_length_Short,trip_length_Medium,trip_length_Long,trip_length_Very Long
0,SpiceJet,SG-8709,Delhi,0,Night,Mumbai,0,130,71.44,0,0,1,0,0,0,1,0,0,0
1,SpiceJet,SG-8157,Delhi,0,Morning,Mumbai,0,140,71.44,0,1,0,0,0,0,1,0,0,0
2,AirAsia,I5-764,Delhi,0,Early_Morning,Mumbai,0,130,71.47,0,1,0,0,0,0,1,0,0,0
3,Vistara,UK-995,Delhi,0,Afternoon,Mumbai,0,135,71.46,0,0,0,0,1,0,1,0,0,0
4,Vistara,UK-963,Delhi,0,Morning,Mumbai,0,140,71.46,0,0,0,0,1,0,1,0,0,0


In [38]:
#drop cloumns? fligh, duration, and deptaure class